In [1]:
import re
import string
import pygtrie
import math
import numpy as np
import operator 

In [2]:
#Opens the document vectors to split to test and train
f = open("r.Doc_Vec", "r")
lines = f.readlines()
f.close()

#Opens seperate test and train files to split the DV file.
fTest = open("r.test", "w+")
fTrain = open("r.train", "w+")

#if the value of the counter is odd the data belongs to train class
#if the value of the counter is even the data belongs to test class
dummyCounter =0
for line in lines:
    dummyCounter = dummyCounter+1           #a dummyCounter to track the document ID
    if dummyCounter%2 ==1:                  #if document ID is a odd number I say that it belongs to train set 
        fTrain.write(line)                  #saves the line into the train document
    else:                                   #if document ID is an even number I sat that it belongs to test set
        fTest.write(line)                   #saves the line into the test document

In [3]:
#opens the file to start the training process
f = open("r.train", "r")
lines = f.readlines()
f.close()

dummyDict=dict()
dummyCounter=0

#saves each line of the training data to the dictionary and array dataset
#to make sure more pretentble for me
for line in lines:
    line= re.sub( '\n', '', line)         #trims the "\n" from the line
    instances=line.split("\t")            #splits the data according to the tabs 
    dummyCounter=dummyCounter+1           #a dummy counter to save the document IDs data, it might be useful...
    dummyList=[]                          #a dummy empty array to append the seperated data as array
    for i in range(len(instances)):
        dummyList.append(instances[i])    #appends each and every data into the dummy empty array/list
    dummyDict[dummyCounter]=dummyList     #saves the dummy list to the dictionary by using document ID as the key

In [ ]:
#iterates through the both documents to calculate the similarity between the documents
#the results that we have found after this iteration calculates the disance by using 
#cosine similarity
def findSimilarityBetweenDocs(doc1, doc2):    
    j=0                                   #an empty variable to track the second documents
    for i in range(len(doc1)):            #starts the loop by tracking the variables of first document 
        if i%2==0:                        #since we both stored the frequecy of the terms as well as the term ids there are two variables to use while calculating the cosine similarity
            if doc1[i]<doc2[j]:           #if the term of first document is smaller than the second document
                i=i                       #wait for loop to iterate
            elif doc1[i]>doc2[j]:         #if the term of the first document is larger than the first document
                j=i+2                     #iterate through the second document until find smaller than the first document
            elif doc1[i]==doc2[j]:        #if both terms are same 
                cos_sim(doc1[i], doc2[j]) #calculate the cosine similarity between documents
                j=i+2                     #cahnge the position of the pointer of the second document as well.
                
#Calculates the distance between two vectors by using cosine similarity
def cos_sim(a, b):
    dot_product = np.dot(a, b)                #dot product of the first and second vector
    norm_a = np.linalg.norm(a)                #finds the length of the first vector
    norm_b = np.linalg.norm(b)                #finds the length of the second vector
    return dot_product / (norm_a * norm_b)    #calculates the distance

#find the neighbors of the testing instances
#gets the training set, test instances and k as a parameter to the method
def getNeighbors(trainingSet, testInstance, k):
    distances = []                           #an empty array for storing the distances                                        
    length = len(testInstance)-1             #since we have 0 the lenght should be -1
    for x in range(len(trainingSet)):        #a loop to iterate in the traininf set
        dist = cos_sim(testInstance, trainingSet[x]) #calculates the cosine similarity between the documents
        distances.append((trainingSet[x], dist))     #appends the result of teh distances into the array/list
    distances.sort(key=operator.itemgetter(1))       #sorts the distances in order to pick 5 of them
    neighbors = []                           #an empty array to store the 5 closest neighbours
    for x in range(k):                       #a for loop iterates to the 5 to find the closest neighbours
        neighbors.append(distances[x][0])    #saves those distances to the empty list
    return neighbors                         #returns the 5 neighbours

#calculates the accuracy of the algorithm 
#gets the real results and predicted genres of the documents
def getAccuracy(testSet, predictions):
    correct = 0                              #a dummy counter to store correct predictions
    for x in range(len(testSet)):            #a loop to iterate in test document
        if testSet[x][-1] is predictions[x]: #is the test set is same with the predicted result 
            correct += 1                     #increase the value of the dummy correct variable
    return (correct/float(len(testSet))) * 100.0  #calculates the acc.

similarityDict=dict()
for i in range(len(dummyDict)):              #a loop to find the similarity between each and every document
    dummyList=[]                             #a dummy list to store the distances
    dummyList.append(findSimilarityBetweenDocs(dummyDict[i], dummyDict[i+1])) #finds the similarity
    similarityDict[i] = dummyList

k = 5                                         
neighbors = getNeighbors(trainSet, testInstance, k) #finds the neighbours
accuracy = getAccuracy(testSet, predictions)        #gets the accuracy

In [ ]:
#for each and every type of document the rocchio algorithm finds centroids
    def train(train, token_pool):
        for term, data in trainDict.items():
            for cl in trainDict:
                if cl in data:
                    centroid += data[cl]

        for cl in class_labels:
            centroids.append(np.zeros((len(self.tdict), 1)))
            for doc in token_pool[cl]:
                centroids[train_dict[cl]] = vec

            self.centroids[self.lbl_dict[cl]] /= len(token_pool[cl])

    def predict(self, doc):
        distances = []
        for i in range(self.k):
            distances.append(np.linalg.norm(doc_vec - self.centroids[i]))
        return self.class_labels[distances.index(min(distances))]

    tdict = createDictionary(class_titles, pool)
    rocchio = Rocchio(class_titles, tdict)
    rocchio.train(pool)
    
    for c in rocchio.centroids:
        print np.linalg.norm(c - rocchio.centroids[0])
    id = 3
    lbl = rocchio.predict(tokenizeDoc(test[class_titles[id]][3]))
    test_lbl_pool = rocchio.predictPool(test_pool)
    metrics = calculateMetrics(class_titles, test_lbl_pool)
    total_F = 0
    for cl in class_titles:
        Acc = ((metrics[cl]["tp"] + metrics[cl]["tn"])* 1.0 / (metrics[cl]["tp"] + metrics[cl]["fp"] + metrics[cl]["fn"] + metrics[cl]["tn"]))
        print 'accuracy = ', Acc
        